In [1]:
!pip install --upgrade transformers

In [2]:
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

In [3]:
!pip install kaggle

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"kagglecomabubakar","key":"1ec7345524fb7ca73567796c2be50455"}'}

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip install opendatasets

In [7]:
from opendatasets import download
download('https://www.kaggle.com/datasets/nileshmalode1/samsum-dataset-text-summarization')

Skipping, found downloaded files in "./samsum-dataset-text-summarization" (use force=True to force download)


In [8]:
import os
path = os.listdir('/content/samsum-dataset-text-summarization')
path

['samsum_dataset',
 'samsum-train.csv',
 'samsum-test.csv',
 'samsum-validation.csv']

In [9]:
train_data = pd.read_csv('/content/samsum-dataset-text-summarization/samsum-train.csv')

validation_data = pd.read_csv('/content/samsum-dataset-text-summarization/samsum-validation.csv')

In [10]:
train_data.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [11]:
train_data.shape

(14732, 3)

In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14732 entries, 0 to 14731
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        14732 non-null  object
 1   dialogue  14731 non-null  object
 2   summary   14732 non-null  object
dtypes: object(3)
memory usage: 345.4+ KB


In [13]:
train_data.isnull().sum()

,0
id,0
dialogue,1
summary,0


In [14]:
train_data.dropna(inplace=True)

In [15]:
train_data.isnull().sum()

,0
id,0
dialogue,0
summary,0


In [16]:
def show_row_details(dataset, idx):
    """
    Show all details of a single row in a HuggingFace Dataset.

    Args:
        dataset : HuggingFace Dataset
        idx     : row index (int)
    """
    row = dataset.iloc[idx]  # get row
    print(f"Row index: {idx}\n")


    for key, value in row.items():
      print(f"{key}: {value}\n")

# Example usage:
show_row_details(train_data, 	0)


Row index: 0

id: 13818513

dialogue: Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)

summary: Amanda baked cookies and will bring Jerry some tomorrow.



In [17]:
train_data = train_data.sample(n=4000,random_state=42).reset_index(drop=True)
validation_data = validation_data.sample(n=500, random_state=42).reset_index(drop=True)

In [18]:
# Clean the text by removing unwanted characters
import re

def clean_text(text):
    text = re.sub(r'\r\n', ' ', text)  # Remove carriage returns and line breaks
    text = re.sub(r'\s+', ' ', text)    # Remove extra spaces
    text = re.sub(r'<.*?>', '', text)  # Remove any XML tags
    text = text.strip().lower()        # Strip and convert to lower case
    return text

# Apply cleaning to dialogue and summary columns
train_data['dialogue'] = train_data['dialogue'].apply(clean_text)
train_data['summary'] = train_data['summary'].apply(clean_text)

validation_data['dialogue'] = validation_data['dialogue'].apply(clean_text)
validation_data['summary'] = validation_data['summary'].apply(clean_text)


# Display a sample after cleaning
train_data

,id,dialogue,summary
0,13811908,violet: hi! i came across this austin's articl...,violet sent claire austin's article.
1,13731387,"mel: right, it's time i bought some christmas ...",mel bought christmas presents while evie keeps...
2,13727910,carol: can you pls pick up when i call you. ca...,"fiona can't talk to carol, because she's very ..."
3,13729823,"adam: do u have a map of paris? tom: yes, why?...",tom has a map of paris.
4,13681400,"frank: hi, how's the family? mike: great! sam'...","mike is happy, because sam's moved out. mike a..."
...,...,...,...
3995,13681041,barry: hello buddy michael: hey barry: do you ...,barry and michael will watch football instead ...
3996,13821525,piotr: jackie: haha omg trump may: the world ...,"may, jackie and piotr are laughing at trump."
3997,13680433,logan: what are u wearing? emily: what?! i’m a...,emily is at work wearing her work clothes. emi...
3998,13812716,emma: did you finish the book i gave you? liam...,"emma gave ""the first fifteen lives of harry au..."


In [20]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [21]:
#length of text using tokenizer method

input_max_len = max(len(tokenizer.encode(text)) for text in train_data['dialogue'])
output_max_len = max(len(tokenizer.encode(text)) for text in train_data['summary'])

input_max_len, output_max_len

Token indices sequence length is longer than the specified maximum sequence length for this model (740 > 512). Running this sequence through the model will result in indexing errors


(1224, 103)

In [22]:
#length of text without toekenizer

input_max_len = max(len(text) for text in train_data['dialogue'])
output_max_len = max(len(text) for text in train_data['summary'])

input_max_len, output_max_len

(4143, 300)

In [23]:
# Preprocessing function for tokenization
def preprocess_function(examples):
    # Tokenize the dialogue and summary
    inputs = tokenizer(examples["dialogue"], padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(examples["summary"], padding="max_length", truncation=True, max_length=150)
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply the preprocessing
train_dataset = train_data.apply(preprocess_function, axis=1)
val_dataset = validation_data.apply(preprocess_function, axis=1)

In [24]:
train_dataset[0]

{'input_ids': [25208, 10, 7102, 55, 3, 23, 764, 640, 48, 403, 17, 77, 31, 7, 1108, 11, 3, 23, 816, 24, 25, 429, 253, 34, 1477, 25208, 10, 3, 7997, 15, 10, 7102, 55, 3, 10, 61, 2049, 6, 68, 3, 23, 31, 162, 641, 608, 34, 5, 3, 10, 61, 3, 7997, 15, 10, 68, 2049, 21, 1631, 81, 140, 3, 10, 61, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## **Fine Tunning**

In [26]:

model_dir='/content/drive/MyDrive/saved_t5_hugging_face_models'



os.makedirs(model_dir, exist_ok=True)

In [31]:
# Model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Define training arguments
training_args = TrainingArguments(
    output_dir=model_dir,          # output directory for checkpoints
    num_train_epochs=6,              # number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir="./logs",            # directory for storing logs
    logging_steps=50,                # how often to log training info
    save_steps=500,                  # how often to save a model checkpoint
    eval_steps=50,                   # how often to run evaluation
    eval_strategy="epoch",     # Ensure evaluation happens every `epoch`
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hurairakhan123456789 (hurairakhan123456789-governament-postgraduate-college-ma) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.422300,0.379478
2,0.389000,0.359023
3,0.376200,0.353969
4,0.354800,0.350245
5,0.371900,0.349188
6,0.356200,0.349367


TrainOutput(global_step=3000, training_loss=0.91300408299764, metrics={'train_runtime': 1226.1921, 'train_samples_per_second': 19.573, 'train_steps_per_second': 2.447, 'total_flos': 3248203235328000.0, 'train_loss': 0.91300408299764, 'epoch': 6.0})

## **Save and load model**

In [32]:
# Save the fine-tuned model
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

('/content/drive/MyDrive/saved_t5_hugging_face_models/tokenizer_config.json',
 '/content/drive/MyDrive/saved_t5_hugging_face_models/special_tokens_map.json',
 '/content/drive/MyDrive/saved_t5_hugging_face_models/spiece.model',
 '/content/drive/MyDrive/saved_t5_hugging_face_models/added_tokens.json')

## **Load Model**

In [33]:
# Load the saved model and tokenizer
model = T5ForConditionalGeneration.from_pretrained(model_dir)
tokenizer = T5Tokenizer.from_pretrained(model_dir)

## **Summarization System**

In [49]:

# Ensure the model is on the correct device (GPU if available)
device = model.device  # Get the device the model is on

def summarize_dialogue(dialogue):
    dialogue = clean_text(dialogue)  # Assuming clean_text is defined
    inputs = tokenizer(dialogue, return_tensors="pt", truncation=True, padding="max_length", max_length=512)

    # Move input tensors to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}




    # Generate summary
    outputs = model.generate(
        inputs["input_ids"],
        max_length=150,
        num_beams=4,
        early_stopping=True
    )

    # Decode the generated summary
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [50]:
# Test with a sample input
sample_dialogue = """
Violet: Hey Claire! I was reading an article about Austin and thought you might find it interesting!
Violet: It's about the current trends in urban development and how cities are planning for the future.
Violet: Here, let me share the link: <file_other>
Claire: Oh wow, that sounds like an insightful read. But I've actually already read that one last week.
Claire: It was really interesting though, especially the part about sustainable architecture in cities.
Claire: You know, I've been following these urban planning discussions for a while now.
Violet: Oh, I didn’t know that! Well, I’ll look for something else then, maybe something about eco-friendly cities or tech innovations.
Claire: That would be awesome! Let me know if you find something cool.
Violet: Sure, I’ll keep you posted. Thanks for the feedback!
"""

summary = summarize_dialogue(sample_dialogue)
print("Summary:", summary)

Summary: claire was reading an article about austin and thought she might find it interesting. violet has already read that one last week.


In [41]:

# Test with a dialogue on a current news topic
sample_dialogue = """
Alice: Hey John, have you heard about the new project at work?
John: Yes — I saw the email yesterday. It sounds interesting but a bit challenging.
Alice: They want us to redesign the website interface and also implement a better search feature.
John: That’s going to take a lot of time. Do they expect us to finish it by next month?
Alice: I think the deadline is six weeks from now. But with the new interns, we might manage.
John: True. Also, the project manager said we should meet tomorrow to plan the tasks.
Alice: Ok, let’s prepare a task list tonight. We need to divide responsibilities and set priorities.
John: Agreed. I’ll start drafting the list and share with you.
Alice: Great. If we start early and work together, we can complete it well before deadline.
John: Sounds good. I’m on board.

"""

summary = summarize_dialogue(sample_dialogue)
print("Summary:", summary)

Summary: john and alice have heard about the new project at work. they want them to redesign the website interface and implement a better search feature. they expect them to finish it by next month.
